In [1]:
import pandas as pd
import jieba

import jieba.posseg as pseg
jieba.set_dictionary('jieba/dict.txt')

jieba.load_userdict('userdict/symptom.txt')
jieba.load_userdict('userdict/time.txt')
jieba.load_userdict('userdict/position.txt')
jieba.load_userdict('userdict/others.txt')
jieba.initialize()

import re
from collections import defaultdict
import matplotlib.pyplot as plt

import time as timer
start_time = timer.time()

position_replace = {
    #'疼痛':'痛','疼':'痛','痠痛':'痛','脹痛':'痛','腫痛':'痛','絞痛':'痛','悶痛':'痛',
    '腹部':'腹','肚子':'腹',
    '胸部':'胸','胸口':'胸',
    '肩胛骨':'背','背部':'背',
    '臉部':'臉','臉頰':'臉','鼻子':'臉','耳朵':'臉','唇':'臉','眼':'臉','嘴唇':'臉','額頭':'臉',
    '脖子':'頸','頸部':'頸','後頸':'頸',
    '嘴巴':'嘴','牙齦':'嘴','口腔':'嘴','唇':'嘴',
}

sym_replace = {
    '疼痛':'痛','疼':'痛','痠痛':'痛','脹痛':'痛','腫痛':'痛','絞痛':'痛','悶痛':'痛',
    '嘔吐':'吐','想吐':'吐',
    '心跳快':'心悸',
    '麻木':'麻',
    '發冷':'畏寒',
    '疹子':'紅疹',
}

f_replace = {
    '左邊':'',
    '右邊':'',
    '左':'',
    '右':'',
    '左方':'',
    '右方':'',
    '雙':'',
}


kcom = pd.read_csv('Kcom_dr568410605emgcase.csv', lineterminator='\n')
kcom['Kcom'] = kcom['Kcom'].fillna("並無主訴")

print(len(kcom))
kcom.head()

Building prefix dict from /Users/kuanwen/Desktop/自然語言/project/jieba/dict.txt ...
Loading model from cache /var/folders/tv/yy9kh1h15dsfg0vcy7vbgtmw0000gn/T/jieba.u936d01f3f8b35df8fcf902039c509f73.cache
Loading model cost 1.158 seconds.
Prefix dict has been built succesfully.


241822


,Pno,Caseno,Kcom
0,15929545,"5,621",今天下午開始雙腳抽筋 現求治
1,15941825,"5,011",剛剛在家全身不適 臉潮紅 在家量血壓高 現求治
2,15965603,"6,001",據家屬訴今天早上開始食慾差 不說話 全身無力 現意識不清 119測血糖33MG/DL
3,15966387,"6,801",昨天開始頭暈不適 現未改善
4,15968549,"5,441",昨天開始左腰 左腹痛 想吐 外院檢查有結石 現持續中


In [2]:
max_length = 0

def replace_w(string, replace_dict):
    fin = 0
    
    while fin == 0:
        match = list()
        
        for d in replace_dict:
            if d in string:
                match.append(d)
        
        if len(match) == 0:
            fin = 1
        else:
            max_seg = ''
            for m in match:
                if len(m) > len(max_seg):
                    max_seg = m
            
            string = string.replace(max_seg, replace_dict[max_seg])
    return string

with open('kcom_output_0611.csv','w', encoding="utf_8_sig") as the_file:
    the_file.write('Pno,Caseno,')
    for k in range(1,10):
        the_file.write('Kcom_'+str(k)+',Kcom_du'+str(k)+',')
    the_file.write('Kcom_10,Kcom_du10')
    the_file.write('\n')

    for i in range(len(kcom)):
        split = kcom['Kcom'].loc[i].replace("有","")
        split = split.replace("會","")
        split = split.split()


        t_now = 0
        m = 0
        pre_time = list()
        event_w = list()
        event_now = list()

        for s in split: 
            t_w = list()
            sym_w = list()
            pos_w = list()

            last_t = 0
            last_n = 0
            last_f = 0
            last_sym = 0
            t = 0
            sym = 0



            line = pseg.cut(s,HMM=False)
            for k,f in line:
                if f == 't':
                    last_n = 0
                    last_sym = 0
                    last_f = 0
                    if t_now == 0:
                        if sym == 1:
                            event = dict()
                            event['time'] = t_w
                            event['sym'] = sym_w
                            event_w.append(event)
                            t_w = list()
                            sym_w = list()
                            pos_w = list()
                            sym = 0
                            t_w.append(k)
                            last_t = 1
                            t = 1  
                        elif last_t == 1 and t == 1:
                            t_w.append(k)
                            last_t = 1
                            t = 1
                        elif t == 0:
                            t_w.append(k)
                            last_t = 1
                            t = 1

                elif f == 'tnow' or f == 'tg':
                    last_n = 0
                    last_t = 0
                    last_sym = 0
                    last_f = 0
                    t_now = 1
                    if sym == 1:
                        event = dict()
                        event['time'] = t_w
                        event['sym'] = sym_w
                        event_w.append(event)
                        t_w = list()
                        sym_w = list()
                        pos_w = list()
                        sym = 0


                elif f == 'sym':
                    if last_n == 1:
                        for p in pos_w:
                            p = replace_w(p, position_replace)
                            p = replace_w(p, f_replace)
                            k = replace_w(k, sym_replace)
                            sym_w.append(p+k)
                        pos_w = list()
                    else:
                        k = replace_w(k, sym_replace)
                        sym_w.append(k)

                    last_n = 0
                    last_t = 0
                    last_sym = 1
                    last_f = 0
                    sym = 1

                elif f == 'f':
                    last_f = 1
                    last_sym = 0
                    last_n = 0
                    last_t = 0
                    n = k
                elif f == 'pos':
                    if last_f == 1:
                        #n = replace_w(n, f_replace)
                        n = n+k
                        pos_w.append(n)
                    elif last_n == 1:
                        pos_w.append(k)
                    else: 
                        n = k
                        pos_w.append(n)

                    last_n = 1
                    last_sym = 0
                    last_f = 0
                    last_t = 0
                elif f == 'm' and last_sym == 1:
                    last_t = 0
                    last_f = 0
                    last_n = 0
                    last_sym = 0
                    if len(t_w) == 0:
                        t_w.append(k)
                        m = 1
                        last_t = 1

                else:
                    last_f = 0
                    last_t = 0
                    last_n = 0
                    last_sym = 0

            if len(sym_w) > 0:
                if t_now == 1:
                    event_now += sym_w
                else:
                    event = dict()
                    if len(t_w) == 0:
                        event['time'] = pre_time
                    else:
                        event['time'] = t_w 
                    event['sym'] = sym_w
                    event_w.append(event)
            elif len(t_w) > 0:
                    pre_time = t_w

        total = list()
        
        time = ''
        for e in event_w:
            event = dict()
            #print(e)
            
            if len(e['time']) > 0:
                time = ''.join(e['time'])
            
            if len(e['sym']) > 1:
                for esym in e['sym']:
                    event = dict()
                    event['time'] = time
                    event['sym'] = esym
                    total.append(event)
                    
            elif len(e['sym']) == 1:
                event['time'] = time
                event['sym'] = e['sym'][0]

                total.append(event)
                
        if m == 1:
            m_t = ''
            for idx, e in reversed(list(enumerate(total))):
                if len(e['time']) > 0:
                    m_t = e['time']
                else:
                    e['time'] = m_t
        
        if len(event_now) > 0:
            for e_now in event_now:
                total.append({'time': '現在', 'sym': e_now})
                
        
        #print(total)
        
        
        the_file.write(str(kcom['Pno'].loc[i])+','+'"'+str(kcom['Caseno'].loc[i])+'",')

        total_len = len(total)
        
        if total_len > max_length:
            max_length = len(total)
        
        
        
        for k in range(0, total_len-1):
            the_file.write(total[k]['sym'])
            the_file.write(',')
            the_file.write(total[k]['time'])
            the_file.write(',')
    
        if total_len > 0:
            the_file.write(total[total_len-1]['sym'])
            the_file.write(',')
            the_file.write(total[total_len-1]['time'])
            for k in range((10-total_len)*2):
                the_file.write(',')
        else:
            for k in range(19):
                the_file.write(',')

        the_file.write('\n')  
print(max_length)

10


In [3]:
print("--- %s seconds ---" % (timer.time() - start_time))

--- 75.4996600151062 seconds ---
